In [12]:
import duckdb
import pandas as pd

In [14]:
sql_query = '''
select * from "nyc_parking_violations_2"."main_dbt_test__audit"."violation_codes_revenue"
'''

with duckdb.connect('data/nyc_parking_violations_2.db') as con:
    display(con.sql(sql_query).df())

,violation_code,total_revenue_usd
0,41,0.0


In [16]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto('data/dof_parking_violation_codes.csv',
normalize_names= True
)
'''
sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto('data/parking_violations_issued_fiscal_year_2023_sample.csv',
normalize_names= True
)
'''

with duckdb.connect('data/prod_nyc_parking_violations_2.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)

In [17]:
sql_query = '''
show tables
'''

with duckdb.connect('data/prod_nyc_parking_violations_2.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,gold_ticket_metrics
3,gold_vehicles_metrics
4,parking_violation_codes
5,parking_violations_2023
6,silver_violation_tickets
7,silver_violation_vehicles
